# Sentiment Analysis on Twitter tweets using LSTM and Keras
<hr>

### Steps
<ol type="1">
    <li>Load the dataset (13k twitter tweets with manually marked label)</li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Tweets</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>

<hr>
<i>Import all the libraries needed</i>

In [14]:
!pip install Sastrawi
!pip install tensorflow

In [168]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re, io, json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Indonesian Stemmer
import tensorflow as tf 

<hr>
<i>Preview dataset</i>

In [16]:
data = pd.read_csv('utf8_dataset.csv')

print(data[['Tweet', 'HS']])

                                                   Tweet  HS
0      - disaat semua cowok berusaha melacak perhatia...   1
1      RT USER: USER siapa yang telat ngasih tau elu?...   0
2      41. Kadang aku berfikir, kenapa aku tetap perc...   0
3      USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...   0
4      USER USER Kaum cebong kapir udah keliatan dong...   1
...                                                  ...  ..
13164  USER jangan asal ngomong ndasmu. congor lu yg ...   1
13165                       USER Kasur mana enak kunyuk'   0
13166  USER Hati hati bisu :( .g\n\nlagi bosan huft \...   0
13167  USER USER USER USER Bom yang real mudah terdet...   0
13168  USER Mana situ ngasih(": itu cuma foto ya kuti...   1

[13169 rows x 2 columns]


<hr>
<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the Indonesian stop words</i>

In [17]:
indonesian_stopwords = pd.read_csv('stopwords.txt', sep="\n")
indonesian_stopwords = indonesian_stopwords.iloc[:, 0].values.tolist()
indonesian_stopwords

['adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut',
 'beri

Replace alay words

In [18]:
alay_words = pd.read_csv('alay.csv')
# alay_words = alay_words.set_index("alay")
alay_words

row = alay_words[alay_words.alay == "3x"]
# row.empty

In [19]:
# row
print(str(row['replacement'].values[0]))

tiga kali


<hr>

### Load and Clean Dataset

In the original dataset, the tweets are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the tweets by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the tweets.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [20]:
def stemmer(text):
    # Init indonesian stemmer
    factory = StemmerFactory()
    s = factory.create_stemmer()
    result = s.stem(text)
    print(result)
    return result

def remove_stopwords(tweet):
    output = []
    words = tweet.split()
    for word in words:
      if word not in indonesian_stopwords:
        output.append(word)

    return ' '.join(output)

def replace_alay(tweet):
    output = []
    words = tweet.split()
    for word in words:
      row = alay_words[alay_words.alay == word]
      if row.empty:
        output.append(word)
      else:
        output.append(str(row['replacement'].values[0]))

    return ' '.join(output)

    

def load_dataset():
    df = pd.read_csv('utf8_dataset.csv')

    # Remove \n \t \r
    df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

    # Tweets/Input
    x_data = df['Tweet']
    
    # Sentiment/Output
    y_data = df['HS']

    # PRE-PROCESS TWEETS
    x_data = x_data.apply(lambda tweet: tweet.lower())

    # Remove HTML tags
    x_data = x_data.replace({'<.*?>': ''}, regex = True)

    # Remove non alphabets
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)

    # Remove words that is lees than 2 chars
    x_data = x_data.apply(lambda tweet: ' '.join([w for w in tweet.split() if len(w) > 2]))

    # Remove RT
    x_data = x_data.str.replace('rt', '')

    # Remove USER
    x_data = x_data.str.replace('user', '')

    # Remove URL
    x_data = x_data.str.replace('url', '')

    # Remove excess spaces
    x_data = x_data.apply(lambda tweet: ' '.join(tweet.split()))

    # Trim
    x_data = x_data.str.strip()
    
    # Remove stop words
    x_data = x_data.apply(lambda tweet: remove_stopwords(tweet))

    # Replace alay words
    x_data = x_data.apply(lambda tweet: replace_alay(tweet))

    # Stem
    x_data = x_data.apply(lambda tweet: stemmer(tweet))
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace(1, 1)
    y_data = y_data.replace(0, 0)

    return x_data, y_data

x_data, y_data = load_dataset()

di saat cowok usaha lacak perhati gue kamu lantas remeh perhati gue kasih khusus kamu basic kamu cowok bego
telat beri tau kamu edan sarap gue gaul cigax jifla cal itu licew
kadang pikir percaya tuhan jatuh kali kali kadang tuhan tinggal orang tua rencana pisah kakak pilih kristen anak ter
tau mata sipit lihat
kaum cebong kafir sudah lihat dongok dungu haha
bani taplak dan kawan kawan
deklarasi pilih kepala daerah aman anti hoaks warga dukuh sari jabon
gue saja selesai watch aldnoah zero kampret memang akhir karakter utama cowok kena friendzone bro
admin belanja po baik nak makan ais kepal milo ais kepal horlicks cendol toping kau kau doket mana gerai rozak meuaku taipan kembar baby amp romantika bank islam senawang
enak kalau sambil ngewe
jari belum ukur nyali bacot
banci kaleng malu peanyaan nyungsep koe
kalau ajar ekonomi mesti jago privatisasi hati orang aduh ironi
aktor huru hara prabowo lengser perintah jokowi nyata
guru enak jablay guru sekolah dasar sih kayak nikmat pecun guru


tolol sudah tau perintah rezim komunis bobrok koruptor pimpin sidang dewan wakil rakyat paya cebong
mantan mantan rasa sedih sampai pikir tidak bisa tidur lo rasa anak berengsek saja laki
cina usak bangsa usir tuju
ganti presiden cara konstitusional hak rakyat woiii begini nih kalau plonga plongo laku otoriter
mari wujud pilih kepala daerah kota bengkulu aman damai cok adem heru
serius presiden jokowi kenapa ada islam nusantara kenapa tidak ada kristen nusantara buddha hindu kong hucu nusantara
sama nyata moeldoko bukti istana takut bangkit umat islam
cowok jago drasex ngentot adik luvian
kroasia kafir tau apa ajar kafir sudah lupa kapling sorga
lah nya menteri suhao ada homo
gue tau lah culun takut orang ubah perlahan buruk
perang salib raja afrika bagi babilonia peluk kristen tobaco peluk islam
kejar setor bangsat rupiah turun
moga ada tindak lapor nista agama adek armando langsung ketua front bela islam shobri lubis badan reserse kriminal markas besar polisi republik indonesia gambi

ingat facebook anjir lupa
kalau aku sih bilang alah kontol
nikita mirzani sudah pakai hijab bicara cebang cebong sudah fantasi dia belum cokorers
tatap alih arah enggan tatap pria curiga lagi pula peama kali gaduh suasana angie temu ada pria asing tidur sofa jadi pei
lahir kembang gera komunis indonesia pisah organisasi nama indische sociaal democratische vereeniging
shaheizy sama wajah baharu calpis
serah saja balas fokus kerja tugas proyek seni budaya telah langsung kumpul presentasi syukur nilai muas jadi proyek
bro
laku bodoh masuk kategori buat senang laku sontoloyo menganggu satu bangsa laku naif langsung bagai bentuk intimidasi hadap ulama laku
monyet turun mukim warga cibadak sukabumi
kotor bahagia warga cebong ngei
giveaway taplak meja menang the rich chenle
buka busuk kubu anies sandi menang berkah prabowo usaha cuci tangan
wkwk bodoh bang
bukan kayak orang bego bro ego
menteri lingkung hidup dan hutan upaya perata ekonomi tora kawasan hutan hutan sosial tribun medan
mahfud p

bas teknologi digital presiden sistem jalan langsung ramah guna ratas
ah asa untuk hibur ada cebong tidak bahagia presiden cebong tol cebong lah kamu germo
resah presiden hidup penuh drama citra main film televisi saja joko judul
agama kristen golong darah san fernando valley
buddha bunuh muslim myanmar
yahudi bangkai jerman ibadah
papa minta ketua fraksi rakyat nya setya novanto dewan wakil rakyat pecat ketua dewan wakil rakyat pecat ada dewan wakil rakyat
cina
hoaks kodok biawak roket sekop bangsat karbitan munafik ganti presiden presidenbaru tenggelamkanjokowikesungaisiak
ribut komunis bla bla emoh baca baca kalau niat agitasi serang perintah pakai cari simpatisan
presiden jokowi harap skema kerja bangun bangun
pilih umum langsung biar buat pilih presiden dewan wakil rakyat calon presiden biar tidak kayak kucing dalam karung mungkin bikin mekanisme calon presiden tentu gubernur prestasi maju calon presiden sudah gubernur
tidak vital bagaimana sudah inter tidak bek kanan beda era mai

memang kiai staquf sebut suku ras biar mudah paham nalar kamu bahlul saya kamu arab kamu sembah saja saya sodor arab bani saud pro israel kamu pura
laku dosa anis sandi sampai bully rendah banyak orang jahat licik benar bully rendah dunia kejam kawan
kalau ajar agama buddha hindu benar begini coba ajar benar buka mulut
alam kerja pak time cina best kerja melayu haha
tidak kuat karena jijik cepat asa kejar ya sampai napas saja sungkan wkwk
bubar dewan wakil rakyat bubar save komisi berantas korupsi
hehe pasar global tidak batas lokal masih ada asing asing jujur
gue diving mampus junedi
suciwati revolusi mental jokowi omong kosong bual tuju amp komen
marah panggil sipit hoshi suka mata sepele fisik pakai definisi hoshi milik mata sipit deh
otak makelar jual aset negara maling aset negara peamina jual isi tenaga kerja asing tiongkok ganti presiden
akun siti fatimah amp komen laman ringkas saja parah amp bejat malu moga sehat sabar amp semangat suara benar
bawang putih sedang kafir haram h

...

apa kamu think memang tidak komentar negara agama islam agama minoritas jerman hati sedih dengar larang terap jerman
kaum zionis sontoloyo
sarap tega banget sih laku anak surat pemberitahuan tahun hewan kalau gue suami gue ambil anak gue lapor polisi amp gue cerai langsung cih najis perempuan kayak begitu
gue harun yahya ilmuwan yahudi jaman lah
usir fahri maling maling dewan wakil rakyat
bikin rusuh negara usaha tambang asing masuk
alhamdulillah sadar kalau cebong pintar lucu beda banget kampret goblok beringas wkwk
larang wakil agen jual reksa dana
terima kasih presiden jokowi moga listrik tambah pulau nias manfaat tingkat ekonomi masyarakat nias suara pulau untuk jokowi salam periode
oknum polisi aniaya ibu lagi mabuk mak habis minum kencing onta
selamat lahir teman gera mahasiswa islam indonesia
belum masak bangun hitung konstruksi ide study layak dana bebas lahan dan lain lain tidak hitung
goblok kecebong aku pancasila negara satu republik indonesia otak hati komunis rezim anti ko

lucu kalau ada orang polisi karena tahu sebut cara spesifik kitab agama teentu panggang api
kalau sudah duit ingin banget rumah bikin zen garden belah teras rumah ala jepang begitu kayak rumah bos yakuza ingin banget nongkrong tidur nikmat angin sepoy ngelus kucing gendut muka bodoh
anjing maju babi
dagang sumber jemput tolak hoaks sukses pilih kepala daerah kota diri
karena isu jokowi kafir antek asing antek asing tidak mempan isu partai komunis indonesia alat isu
iya benar khianat partai komunis indonesia
ya bego idiot bawa asa bawa asa mania
hebat benar nih wakil gubernur banget saya suka foto dengan instagram
cowok miss atau queen duit jadi transgender biar miss atau queen etik sih
muslim menteri agama wajib bela penting bangsa negara
begini masalah tenaga kerja asing ilegal cina sudah faktual bukti apa lagi sudah bahas perintah niat selesai
iya tai
kali saksi topeng monyet kampung iri kelompok topeng monyet sulap aksi siksa ekor monyet tonton hibur warga
panas pasang calon kritik 

tidak beda capai rezim jabat negara dewan wakil rakyat tidak ada perintah anggap negara salah tahun
mamah nenek cerita mamah habis beli kerudung lihat nenek lebar kerudung itu gede enak pakai iya taplak meja tertawa sengakak ngakaknya kenap
orang gila saja percaya kalau kelas langsung tel cebong otak
muncul nih sakit nya jam malam kacrut telpon telepon mati kacrut banget sudah
kalau gubernur ahok eks terus ada bayi debora hem rasa
heran setan iblis kabur takut kalau dengar azan makhluk allah anteng
jadi begini sekarang inti orang kayak elite tidak jamin tangan mulut didik lebih baik gue udik kampung sampah masih punya otak pikir
bajing
pilih kepala daerah daerah khusus ibukota penuh intimidasi hasil sah anies sandi kampanye janji janji palsu batal angkat anies sandi
pecun
rezim anti islam pro komunis insyaallah saksi hancur
evi layanto siswi katolik karya yosef istri ronaldo pardede sah
dutee alir komunis ateis tulen pilih presiden filipina mayoritas kristen katolik
hasil revolusi ment

bego enggak beepi daun saja iman langsung luntur
pilih dewan wakil rakyat bubar
memang tidak hujat munafik non
cebong goblok dengan
ih dasar manusia homo
maklum cebong daya ingat paruh
orang bangga pakai itu pacar sama banci anggap banci cewek benar
the lie tju lie tju seroja the lie tju seroja siswi katolik karya yosef istri ronaldo pardede sah
bukti lowong kerja saja kasih asing banyak bohong rakyat
amin ya rabi
tendang pelan asing tetap jokowi
pilih cebong
gue negara sibuk perang dagang perang ayat di sana sibuk mars sibuk ayat dan terus sibuk teriak anti asing dolar minggat mumet blokir demo
minggu mampus saja layan kalau tidak bikin layan bilang saja kualitas kacrut
gue pakai baju buka bilang jablay gue main malam bilang cewek enggak benar kabar hijab hamil lebih dahulu
hubung geblek
goblok benar mata picek
titit kakak ipar
nih baca tai sudah spam kado
presiden ajak ulama sea perintah sejuk suasana jelang pilih kepala daerah serentak
setya novanto pancasila titik temu budaya leluh

adik tengok keliling adik nak cam agama first impression cakap muka cina buddha india hindu melayu islam
makan suruh baca anggar dapat dan belanja negara biar bego lihat itu asumsi harga minyak anggar dapat dan belanja negara pakai angka berapa kenapa ada revisi anggar dapat dan belanja negara jangan cuma ambil berita tetapi analisis susah sih bicara sama anonim belum wkwk
presiden joko widodo jokowi masuk daftar the muslim kepala staf presiden moeldoko sebut predikat bukti jokowi juang penting umat islam tokoh muslim pengaruh themuslim
resep salah gubernur pakkkk presiden presiden got nih lihat kalau tidak percaya
maaf si anu adalah cocok mustafa kemal almarhum
ada buku aktivitas angry burung the film tambah kreativitas tempel stiker warna
nostalgia budaya nos tal gia rindu kadang kadang letak ada kangen sekolah kangen lingkung sekolah yuk nostalgia sekolah
berengsek orang buat bikin gue kasar mulu
kepala polisi daerah babi biadab bodoh tidak otak kali
orang yahudi kristen muslim beem

In [131]:
print('Tweet')
print(x_data, '\n')
print('HS')
print(y_data)

Tweet
0        di saat cowok usaha lacak perhati gue kamu lan...
1        telat beri tau kamu edan sarap gue gaul cigax ...
2        kadang pikir percaya tuhan jatuh kali kali kad...
3                                     tau mata sipit lihat
4          kaum cebong kafir sudah lihat dongok dungu haha
                               ...                        
13164                     bicara ndasmu congor kate anjing
13165                                    kasur enak kunyuk
13166                             hati hati bisu bosan duh
13167    bom real mudah deteksi bom kubur dahsyat ledak...
13168                            situ beri foto kutil onta
Name: Tweet, Length: 13169, dtype: object 

HS
0        1
1        0
2        0
3        0
4        1
        ..
13164    1
13165    0
13166    0
13167    0
13168    1
Name: HS, Length: 13169, dtype: int64


In [132]:
x_data = x_data.str.replace('uniform resource locator', '')
x_data = x_data.apply(lambda tweet: ' '.join([w for w in tweet.split() if not w.startswith('x')]))
x_data = x_data.apply(lambda tweet: ' '.join([w for w in tweet.split() if len(w) > 2]))
x_data

0        di saat cowok usaha lacak perhati gue kamu lan...
1        telat beri tau kamu edan sarap gue gaul cigax ...
2        kadang pikir percaya tuhan jatuh kali kali kad...
3                                     tau mata sipit lihat
4          kaum cebong kafir sudah lihat dongok dungu haha
                               ...                        
13164                     bicara ndasmu congor kate anjing
13165                                    kasur enak kunyuk
13166                             hati hati bisu bosan duh
13167    bom real mudah deteksi bom kubur dahsyat ledak...
13168                            situ beri foto kutil onta
Name: Tweet, Length: 13169, dtype: object

In [163]:
x_data.to_csv('clean-tweet.csv')
y_data.to_csv('clean-hs.csv')

<hr>

### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the tweets and sentiments are in order, where they list positive tweets first and then negative tweets. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [170]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
9618                                        tai banget dit
7613     hai ahok pikir kayak wan abut cetek pikir mere...
9577     damai untuk agama untuk agama kitab suci alqur...
11269    islam sontoloyo anna puasa bulan syawal lebara...
7563        tidak berengsek berisik genit tak tau deh wkwk
                               ...                        
11469    ridwan kamil kembang pesantren usaha rindu jaw...
12891    bijak kembali barang kepada konsumen barang ru...
2235     alhamdulillah presiden banyak kasih contoh ker...
10566          awas provokasi jokowi antek cina anti islam
12930    orang gila saja percaya kalau kelas langsung t...
Name: Tweet, Length: 10535, dtype: object 

10260    hai goblok kabar kemarin admin bilang turki di...
12037                      bank kena virus cebong blangsak
12686               aman pegang kacung babu asing mawoooot
10240                            enak itu ngentot straight
2373                                ava anjing monyet aduh
  

<hr>
<i>Function for getting the maximum tweet length, by calculating the mean of all the tweets length (using <b>numpy.mean</b>)</i>

In [171]:
def get_max_length():
    tweet_length = []
    for tweet in x_train:
        tweet_length.append(len(tweet))

    return int(np.ceil(np.mean(tweet_length)))

max_length = get_max_length()
print(max_length)

72


<hr>

### Tokenize and Pad/Truncate Tweets
A Neural Network only accepts numeric data, so we need to encode the tweets. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the tweets into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each tweets has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all tweets length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [172]:
# ENCODE TWEETS
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum tweets length: ', max_length)
print('Total words: ', total_words)

Encoded X Train
 [[  72   28 3538 ...    0    0    0]
 [ 963   41   85 ...    0    0    0]
 [ 317   69   10 ...    0    0    0]
 ...
 [ 668    4  140 ...    0    0    0]
 [ 739 1393    6 ...    0    0    0]
 [   5  215    8 ...    0    0    0]] 

Encoded X Test
 [[ 963   84 1157 ...    0    0    0]
 [ 503  241 2789 ...    0    0    0]
 [ 195  800  590 ...    0    0    0]
 ...
 [  86   39    7 ...    0    0    0]
 [ 113    9  134 ...    0    0    0]
 [1391 1809  566 ...    0    0    0]] 

Maximum tweets length:  72
Total words:  11476


<hr>

### Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

In [173]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

embed_dim = 128
lstm_out = 196

model = Sequential()
# model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
# model.add(LSTM(LSTM_OUT))


# model.add(
#     tf.keras.layers.Dropout(
#         rate=0.25 # Randomly disable 25% of neurons
#     )
# )

# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.add(Embedding(2000, embed_dim,input_length = x_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

print(model.summary())

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 72, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 72, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm_13 (LSTM)              (None, 196)               254800    
                                                                 
 dense_8 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


<hr>

### Training
For training, it is simple. We only need to fit our <b>x_train</b> (input) and <b>y_train</b> (output/label) data. For this training, I use a mini-batch learning method with a <b>batch_size</b> of <i>128</i> and <i>5</i> <b>epochs</b>.

Also, I added a callback called **checkpoint** to save the model locally for every epoch if its accuracy improved from the previous epoch.

In [174]:
checkpoint = ModelCheckpoint(
    'models/LSTM_2.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [175]:
model.fit(x_train, y_train, batch_size = 128, epochs = 10, callbacks=[checkpoint])

Epoch 1/10


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "C:\ProgramData\Anaconda3\lib\site-packa

<hr>

### Testing
To evaluate the model, we need to predict the sentiment using our <b>x_test</b> data and comparing the predictions with <b>y_test</b> (expected output) data. Then, we calculate the accuracy of the model by dividing numbers of correct prediction with the total data. Resulted an accuracy of <b>86.63%</b>

In [160]:
# y_pred = model.predict_classes(x_test, batch_size = 128)

predict_x = model.predict(x_test, batch_size = 128) 
y_pred = np.argmax(predict_x,axis=1)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

Correct Prediction: 1508
Wrong Prediction: 1126
Accuracy: 57.25132877752468


---

### Load Saved Model

Load saved model and use it to predict a tweet statement's sentiment (positive or negative).

In [30]:
loaded_model = load_model('models/LSTM.h5')

Receives a tweet as an input to be predicted

In [31]:
tweet = str(input('Tweet: '))

Tweet: asd


The input must be pre processed before it is passed to the model to be predicted

In [32]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
tweet = regex.sub('', tweet)
print('Cleaned: ', tweet)

words = tweet.split(' ')
filtered = [w for w in words if w not in indonesian_stopwords]
filtered = ' '.join(filtered)
filtered = [tweet.lower()]

print('Filtered: ', filtered)

Cleaned:  asd
Filtered:  ['asd']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

In [33]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=get_max_length(), padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0]]


This is the result of the prediction which shows the **confidence score** of the tweet statement.

In [34]:
result = loaded_model.predict(tokenize_words)
print(result)

[[0.23105481]]


If the confidence score is close to 0, then the statement is **negative**. On the other hand, if the confidence score is close to 1, then the statement is **positive**. I use a threshold of **0.7** to determine which confidence score is positive and negative, so if it is equal or greater than 0.7, it is **positive** and if it is less than 0.7, it is **negative**

In [35]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative


In [36]:
tokenizer_json = token.to_json()
with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))